In [1]:
!pip install openai sentence-transformers==2.2.2 pypdf SQLAlchemy

In [2]:
import os
import openai
import re
from sentence_transformers import SentenceTransformer
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from openai import OpenAI
from time import perf_counter
import numpy as np
import warnings

In [6]:

def get_embedding(text, embedding_model, out_type="array"):
    text = text.replace("\r", " ").replace("\n", " ")
    embedding = embedding_model.encode(text)
    if out_type == "list":
        return embedding
    elif out_type == "array":
        return np.array(embedding)
        
def similarity_search_from_emb(emb, engine, match_threshold=0.75, match_count=10):
    formatted_str = ', '.join(map(str, emb))
    formatted_str = f"[{formatted_str}]"
    sql = f"""WITH cte AS (SELECT document_domain,document_name, page_number, sequence, text, (embedding_1024 <#> '{formatted_str}') as similarity 
    FROM document_embeddings
    ORDER BY similarity asc
    LIMIT {match_count})
    SELECT * FROM cte
    WHERE similarity < -{match_threshold}"""
    df = None
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        df = pd.read_sql(sql=sql, con=engine)    
    df.similarity *= -1.0
    return df

def similarity_search(text, engine,match_threshold=0.75, match_count=10):
    start = perf_counter()
    #emb = get_embedding(text, embedding_model,out_type="list")
    emb = get_embedding(text, embedding_model,out_type="array")
    end = perf_counter()
    elapsed_time = end - start
    start = perf_counter()
    df = similarity_search_from_emb(emb, engine, match_threshold=0.75, match_count=match_count)
    end = perf_counter()
    elapsed_time = end - start
    return df

def get_surrounding_chunks(engine,document_domain,document_name,sequence, N):
    # SQL query to fetch surrounding chunks
    seq_min = sequence - N
    seq_max = sequence + N
    query = f"""
        SELECT document_domain,document_name,Page_Number,sequence,text  FROM document_embeddings
        WHERE document_domain = '{document_domain}' AND
              document_name = '{document_name}' AND
              sequence BETWEEN '{seq_min}' AND '{seq_max}'
        ORDER BY document_domain,document_name,sequence ASC
    """ 
    result = None
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        result = pd.read_sql(sql=query, con=engine)    
    return result
    

def get_prompt_context_using_rag(query,engine):
    df = similarity_search(query,engine)
    top_rows = df.head(3)
    N = 3
    all_chunks = pd.DataFrame()
    pd.set_option('display.max_colwidth', None)  # For pandas versions 1.0 and later
    for index, row in top_rows.iterrows():
        surrounding_chunks = get_surrounding_chunks(engine,row['document_domain'], row['document_name'], row['sequence'], N)
        all_chunks = pd.concat([all_chunks, surrounding_chunks], ignore_index=True)
    all_chunks.drop_duplicates(subset=['document_name', 'sequence'])
    all_chunks.sort_values(by=['document_name', 'page_number', 'sequence'], inplace=True)
    all_chunks['text'] = all_chunks.sort_values('sequence').groupby(['document_name', 'page_number'])['text'].transform(lambda x: ' '.join(x))
    all_chunks = all_chunks.drop_duplicates(subset=['document_name', 'page_number'])
    all_chunks.reset_index(drop=True, inplace=True)
    prompt_input = all_chunks.apply(lambda row: f"Document: {row['document_name']}, Page: {row['page_number']}, Content: {row['text']}", axis=1).tolist()
    return prompt_input[0]

def get_response(prompt_input,llm_client):
    global messages
    messages = [
                {"role": "system", "content": "You are a helpful assistant.\
                 And will always answer the question asked in 'Question:' and \
                 will quote the Document and Page number at the end of the answer,\
                 the Document: and Page: fields precede the content based on which you will answer."},
                {"role": "user", "content": ''.join(prompt_input)}
          ]

    response = llm_client.chat.completions.create(
                            model = "gpt-3.5-turbo",
                            messages = messages,
                            temperature=0.2,               
                     )
    
    #response_msg = "llm response here"
    response_msg = response.choices[0].message.content
    messages = messages + [{"role":'assistant', 'content': response_msg}]
    return response_msg


def get_answer(query,openai_clien,engine):
    query_string = get_prompt_context_using_rag(query,engine)
    query_string = query_string + f" ques: {query}"
    answer = get_response(query_string,openai_client)
    return answer

In [7]:
document_domain = "RBI_Guidelines"
dbname = 'rbi_bot_db'
dbuser = 'rbi_bot_user'
dbpassword = 'rbi_bot_pwd'
dbhost = '127.0.0.1'
dbport = 5432
chunk_size = 400
embedding_model_name = 'thenlper/gte-large'
embedding_model = SentenceTransformer(embedding_model_name)
db_conn_str = f"postgresql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}"
sql_engine = create_engine(db_conn_str)
openai_client = OpenAI(api_key="nahi_doonga")

In [8]:
print(get_answer("Is there a limit to the interest-rate I can charge a customer on a loan? ",openai_client,sql_engine))
print(get_answer("Can a co-lending partner issue a gold-loan at the customer doorstep?",openai_client,sql_engine))
print(get_answer("Can I host a payment gateway for Indian customers using a server located in Pakistan?",openai_client,sql_engine))
print(get_answer("Can I create a special scheme for SC ST customers?",openai_client,sql_engine))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Question: Is there a limit to the interest rate I can charge a customer on a loan?

Yes, there is no specific limit set by the Bank on the interest rates that can be charged by Housing Finance Companies (HFCs) to customers. However, it is mentioned that interest rates beyond a certain level may be considered excessive and unsustainable, not conforming to normal financial practice. HFCs are advised to have internal principles and procedures in place for determining interest rates, processing charges, and other fees, including penal charges. Transparency in communicating terms and conditions to borrowers is also emphasized.

(Document: FAIRLENDINGPRACTICE1B9DBE75410B4DA881E6EF953304B6F7.pdf, Page: 10)
Question: Can a co-lending partner issue a gold-loan at the customer doorstep?

Yes, a co-lending partner can issue a gold loan at the customer's doorstep, provided certain conditions are met. The loan agreement must contain details of the option to be exercised by the borrower, acceptable 